Here's the data setup from [Project 0](https://cal-cs288.github.io/sp20/projects/proj0.html).



In [0]:
import nltk
import random
import torch

from nltk.corpus import brown
from collections import defaultdict, Counter

nltk.download('brown')
nltk.download('universal_tagset')

brown_tokens = brown.tagged_words(tagset='universal')
print('Tagged tokens example: ', brown_tokens[:5])
print('Total # of word tokens:', len(brown_tokens))

max_word_len = 20

def most_common(s):
    "Return the most common element in a sequence."
    return Counter(s).most_common(1)[0][0]

def most_common_tags(tagged_words, min_count=3, max_len=max_word_len):
    "Return a dictionary of the most common tag for each word, filtering a bit."
    counts = defaultdict(list)
    for w, t in tagged_words:
        counts[w.lower()].append(t)
    return {w: most_common(tags) for w, tags in counts.items() if 
            w.isalpha() and len(w) <= max_len and len(tags) >= min_count}

brown_types = most_common_tags(brown_tokens)
print('Tagged types example: ', sorted(brown_types.items())[:5])
print('Total # of word types:', len(brown_types))

def split(items, test_size):
    "Randomly split into train, validation, and test sets with a fixed seed."
    random.Random(288).shuffle(items)
    once, twice = test_size, 2 * test_size
    return items[:-twice], items[-twice:-once], items[-once:]

val_test_size = 1000
all_data_raw = split(sorted(brown_types.items()), val_test_size)
train_data_raw, validation_data_raw, test_data_raw = all_data_raw
all_tags = sorted(set(brown_types.values()))
print('Tag options:', all_tags)

##

def noun_predictor(raw_data):
    "A predictor that always predicts NOUN."
    predictions = []
    for word, _ in raw_data:
        predictions.append('NOUN')
    return predictions

def accuracy(predictions, targets):
    """Return the accuracy percentage of a list of predictions.
    
    predictions has only the predicted tags
    targets has tuples of (word, tag)
    """
    assert len(predictions) == len(targets)
    n_correct = 0
    for predicted_tag, (word, gold_tag) in zip(predictions, targets):
        if predicted_tag == gold_tag:
            n_correct += 1

    return n_correct / len(targets) * 100.0

def evaluate(predictor, raw_data):
    return accuracy(predictor(raw_data), raw_data)

print('noun baseline validation accuracy:', 
      evaluate(noun_predictor, validation_data_raw))

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
Tagged tokens example:  [('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN')]
Total # of word tokens: 1161192
Tagged types example:  [('a', 'DET'), ('aaron', 'NOUN'), ('ab', 'NOUN'), ('abandon', 'VERB'), ('abandoned', 'VERB')]
Total # of word types: 18954
Tag options: ['ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']
noun baseline validation accuracy: 55.1


The last three letters of a word carry information about the most common part of speech. Here's how you might exploit that fact without a neural network.

In [0]:
from itertools import groupby

# Find all the ends of words in the training set.
last_letters = {w[-3:] for w, _ in train_data_raw}

# For each word ending, find the most common part of speech.
last_letter_tags = {c: most_common([t for _, t in words]) for 
                    c, words in groupby(train_data_raw, key=lambda x: x[0][-3:])}

def last_letter_predictor(raw_data):
    "Guess the most common part of speech based only on the last 3 letters."
    predictions = []
    for word, _ in raw_data:
        # If the end of this word hasn't been seen before, guess 'NOUN'
        predictions.append(last_letter_tags.get(word[-3:], 'NOUN'))
    return predictions

print('last letter validation accuracy:', 
      evaluate(last_letter_predictor, validation_data_raw))


last letter validation accuracy: 76.9


To achieve a similar result with a neural network, we first create data matrices that contain only the last three letters.

In [0]:
def make_matrices(data_raw):
    """Convert a list of (word, tag) pairs into tensors with appropriate padding.
    
    character_matrix holds character codes for each word, 
      indexed as [word_index, character_index]
    character_mask masks valid characters (1 for valid, 0 invalid), 
      indexed similarly so that all inputs can have a constant length
    pos_labels holds part-of-speech one-hot vectors,
      indexed as [word_index, pos_index] with 0/1 values
    """
    # This part is slightly different from the project because we only
    # allocate space for three letters instead of all of them.
    character_matrix = torch.zeros(len(data_raw), 3, dtype=torch.int64) 
    character_mask = torch.zeros(len(data_raw), 3, dtype=torch.float32)

    pos_labels = torch.zeros(len(data_raw), dtype=torch.int64)
    
    for word_i, (word, pos) in enumerate(data_raw):
        # This part is different from the project. Instead of encoding all
        # letters, we just encode the last three.
        last_three_letters = word[:-3:-1]  # reversed, in case of 2-letter words
                                           # E.g., 'dogs' => 'sgo'
        for char_i, c in enumerate(last_three_letters):
            character_matrix[word_i, char_i] = ord(c)

            # The mask is only 1 if there is a letter in the char_i position.
            # For 1-letter and 2-letter words, the character_mask would have 0's.
            character_mask[word_i, char_i] = 1
        pos_labels[word_i] = all_tags.index(pos)
    return torch.utils.data.TensorDataset(character_matrix, character_mask, pos_labels)

validation_data = make_matrices(validation_data_raw)

Here's an illustrated network that makes a prediction based on these three letters. This network is not identical to the lookup-based approach above, but it produces a similar validation accuracy (around 76%).

In [0]:
import torch
from torch import nn
import torch.nn.functional as F

class LastLettersNetwork(nn.Module):
    def __init__(self, n_outputs):
        super().__init__()

        char_types = 128  # An upper bound on the number of unique characters
        # we actually will only use a smaller set of characters, but we are using ascii codes for the character ids that can have values up to 127

        embedding_size = 32  # Arbitrary dimensionality of the embeddings

        # Create a parameter for the embedding vectors for each character.
        # An embedding is a learned position in some vector space that has some
        # arbitrary dimension, such as 32 (small) or 128 (more typical).
        self.emb = nn.Embedding(char_types, embedding_size)

        # Create a projection layer to project three stacked character embedding 
        # vectors into the output space.
        # The second size n_outputs is the number of possible parts of speech.
        self.output_layer = nn.Linear(3 * embedding_size, n_outputs)

    def forward(self, chars, mask):
        # Embed all three characters. Even for 2-letter words, we embed all 3.
        # Now x is a tensor for a batch of words with dimensions that tell you
        # (which example in batch, which character, which element of the embedding)
        x = self.emb(chars)

        # Mask out any characters that aren't there (for 1- and 2-letter words).
        # The "unsqueeze" call is needed to edit the shape of the mask tensor,
        # so that it is compatible with the shape of the embedding tensor.
        #
        # The dimensions of these things are...
        # x:                 (# examples, # characters, hidden_size)
        # mask:              (# examples, # characters)
        # mask.unsqueeze(2): (# examples, # characters, 1)
        #
        # So, x and mask.unsqueeze(2) aren't the same dimension, but they
        # are compatible under broadcasting: 
        # https://pytorch.org/docs/stable/notes/broadcasting.html
        x = x * mask.unsqueeze(2)

        # For each element in the batch, there's a (3, hidden_size) matrix.
        # Instead, build a (hidden_size * 3) vector as a concatenation of all
        # three character vectors. torch.reshape could achieve this too,
        # but torch.cat is handy for the last part of the project. 
        x = torch.cat((x[:,0,:], x[:,1,:], x[:,2,:]), dim=1)

        # It would be reasonable to add more layers, non-linearity and dropout.

        # Project into the output space.
        x = self.output_layer(x)

        
        return x